In [1]:
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
import json
import re
import jsonpath

In [10]:
#先通过观察网页的变化，我们知道了评论使用ajax格式写的，那么他会在一个json接口里
#通过抓包的变化，我们可以获得接口——就是GET后面的那一坨（去掉HTML 1.1）
#通过观察这个url里的参数，发现有个currentPage参数是用来分页的


#https://rate.taobao.com/feedRateList.htm?auctionNumId=574347121195&userNumId=395522079&currentPageNum=2&pageSize=20&rateType=&orderType=sort_weight&attribute=&sku=&hasSku=true&folded=0&ua=098%23E1hve9vbvnpvUvCkvvvvvjiPRLz9zjt8PschAjYHPmPZQjtjR2qyzjiEn2cv0j1n9phv2nQGIVKuzYswzWLj7FyCvvpvvvvv2QhvCvvvMMGCvpvVvvpvvhCvkphvC99vvOC0B4yCvv9vvUm6hK5dxbyCvm9vvvvvphvvvvvv99Cvpvk6vvmmvhCvmhWvvUUvphvUI9vv99CvpvkkmphvLvshL9vjcn1ldE7rV361iNLZ6CeffCuYiXVvVE6Fp%2B0x9Wo4jLEc6acEKBmAVAdUaNLXrqpyCWmQD70OV369mfmxfBAKNoXl%2BExrV4T5vpvhvvmv9IwCvvpvCvvvRphvCvvvvvv%3D&_ksTS=1551875457537_1890&callback=jsonp_tbcrate_reviews_list
item_list=[]

def main():
    url='http://rate.taobao.com/feedRateList.htm?auctionNumId=574347121195&userNumId=395522079&currentPageNum=1&pageSize=20&rateType=&orderType=sort_weight&attribute=&sku=&hasSku=true&folded=0&ua=098%23E1hve9vbvnpvUvCkvvvvvjiPRLz9zjt8PschAjYHPmPZQjtjR2qyzjiEn2cv0j1n9phv2nQGIVKuzYswzWLj7FyCvvpvvvvv2QhvCvvvMMGCvpvVvvpvvhCvkphvC99vvOC0B4yCvv9vvUm6hK5dxbyCvm9vvvvvphvvvvvv99Cvpvk6vvmmvhCvmhWvvUUvphvUI9vv99CvpvkkmphvLvshL9vjcn1ldE7rV361iNLZ6CeffCuYiXVvVE6Fp%2B0x9Wo4jLEc6acEKBmAVAdUaNLXrqpyCWmQD70OV369mfmxfBAKNoXl%2BExrV4T5vpvhvvmv9IwCvvpvCvvvRphvCvvvvvv%3D&_ksTS=1551875457537_1890&callback=jsonp_tbcrate_reviews_list'
    headers={
        'Host': 'rate.taobao.com',
        'Connection': 'keep-alive',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36',
        'Accept': '*/*',
        'Referer': 'https://item.taobao.com/item.htm?spm=a219r.lm872.14.8.73174241dTr9fb&id=574347121195&ns=1&abbucket=8',
#Accept-Encoding: gzip, deflate, br
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Cookie': 'miid=1207089963674790810; t=8d549bb851868992e3cba443e232196b; cna=tRkGFeK+dkICAbTQOpD+JV6p; thw=cn; v=0; cookie2=173359fc98d5e0ad6efefa82490a262e; _tb_token_=f813bae683e77; tk_trace=oTRxOWSBNwn9dPyorMJE%2FoPdY8zfvmw%2Fq5hkapM2bbyhVzsnxk3lBAE2QkgLhBU%2FzZODDFctx89f35m6hdP9bnJ4NF3IMKupSOis9jHvTtL9B54G5nbYArQ6W2xJj6kN%2FIZBDbOqHHQvEBTSAEyVNLU%2FBuG1jYgtDn4iNbSpX37Lpe14MhjvNwWBcKFN%2BCkFONH1bzo1hemCBm9bhzCEQ2hNo0nemslBGYhN%2BNKm7wBus8A%2BOc2Pf1wT3PaqXpP32DXEqdPtqpfzeQzaBjWbCwQfSRrdtA%3D%3D; unb=2742098785; sg=25b; _l_g_=Ug%3D%3D; skt=0cd0753afc690e91; cookie1=B0egshjge8mpqfrpTkkROi7QGhRMPkBEfXPUXUeRNtw%3D; csg=030bc744; uc3=vt3=F8dByEvwbStLoqmlNcw%3D&id2=UU8M8Ji8rub%2Fgg%3D%3D&nk2=DeZZ1UrL66vRW9DtCw%3D%3D&lg2=VFC%2FuZ9ayeYq2g%3D%3D; existShop=MTU1MTg3NTM1NQ%3D%3D; tracknick=naive84107502; lgc=naive84107502; _cc_=WqG3DMC9EA%3D%3D; dnk=naive84107502; _nk_=naive84107502; cookie17=UU8M8Ji8rub%2Fgg%3D%3D; tg=0; enc=v%2BQ56x6oPJiY97QfWVIIpzml%2F8slPkpZVKNdqX9XWIBGzkEo%2F3R08h5YlQlSDz%2FUqnuIIrtFpd1FUz557brZWQ%3D%3D; hng=CN%7Czh-CN%7CCNY%7C156; uc1=cookie16=UtASsssmPlP%2Ff1IHDsDaPRu%2BPw%3D%3D&cookie21=Vq8l%2BKCLjA%2Bl&cookie15=URm48syIIVrSKA%3D%3D&existShop=false&pas=0&cookie14=UoTZ5bsVdXWOOA%3D%3D&tag=8&lng=zh_CN; mt=ci=-1_1; x=e%3D1%26p%3D*%26s%3D0%26c%3D0%26f%3D0%26g%3D0%26t%3D0%26__ll%3D-1%26_ato%3D0; _m_h5_tk=a5182cdf3e67dcc8616aea56f8ec56e8_1551884059064; _m_h5_tk_enc=8fbc2db9780760c3bfc95eb139e78c6f; whl=-1%260%260%261551875701342; l=bBOcP2zqvB8sujdXBOCi5ZsbnAbTDIRVguWfGKNeiO1h5sEyBdoWAJnmo_pJ2Nwla6ZeR3RYGVWyBVTiwyUBl; isg=BNPTCxSsi4WvMUdRXpVtAmamYlf3gAUTnJ2y-4XwVPLgBPGmDVuxmtMSOjTPpL9C',
}
    
    request=urllib.request.Request(url=url,headers=headers)
    json_text=urllib.request.urlopen(request).read().decode()
    #将json格式两边的小括号干掉
    #也可通过string.strip()方法去掉，但是注意，strip可以去掉所有的
    #json_text=json_text.strip('() \n\t\r')   这里是去掉了所有的空格，换行，和括号
    json_text=re.sub(r'\(', '', json_text)
    json_text=re.sub(r'\)', '', json_text)
    
    #将json格式字符串转化为Python对象
    obj=json.loads(json_text)
    
    #现在要抓取内容——用户头像、用户名、评论内容、评论时间、手机类型
    
    #首先找用户评论
    comments_list=obj['comments']
    #遍历这个列表，获取具体的comments信息
    for comment in comments_list:
        #用户头像  我们用jsonpath来提取
        user=jsonpath.jsonpath(comment,'$..avatar')[0]
        face='http:'+user['avatar']
        #用户名
        name=user['nick']
        #评论内容
        ping=comment['content']
        #评论时间
        ping_time=comment['date']
        #手机信息
        info=jsonpath.jsonpath(comment,'..sku')[]
        #将评论信息保存到字典中
        item{
            '用户头像':face,
            '用户名'：name,
            '评论':ping,
            '评论时间':ping_time,
            '用户信息':info,
        }
        item_list.append(item)
    
    
if __name__=='__main__':
    main()
    
    #将获取的内容写成json格式并保存
    string=json.dumps(items_list,ensure_ascii=False)
    with open('ping.txt','w',encoding='utf8') as fp:
        fp.write(string)

URLError: <urlopen error [WinError 10061] 由于目标计算机积极拒绝，无法连接。>